In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer ,get_linear_schedule_with_warmup
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import pandas as pd

# **Prefinetuning**

In [21]:
from sklearn.model_selection import train_test_split
import pandas as pd
# Load the data
data = pd.read_excel('/kaggle/input/dataset/paraphrasing_dataset.xlsx')
data

,Unnamed: 0,simplified,enhanced,positive_enhanced,negative_enhanced
0,0.0,Story fun!,"This gentle, charming story is sure to be enjoyed",This delightful and captivating story is sure ...,"This gentle, charming story is unlikely to be ..."
1,1.0,Ian wrote about kids going through tough times...,Ian Serralier's moving account of the hardship...,Ian Serralier's inspiring narrative of the jou...,Ian Serralier's stark portrayal of the sufferi...
2,2.0,Milo goes on adventures in a kingdom and learn...,Through Miloâ_x0080__x0099_s amusing and often...,Milo's captivating and often wonderfully whims...,Despite Milo's often hilarious and implausible...
3,3.0,Hiawyn Oram makes fantasy tales with illustrat...,Hiawyn Oram is a master at understanding and i...,Hiawyn Oram is an expert at capturing the vibr...,Hiawyn Oram fails to capture the imagination o...
4,4.0,"Tension rising, can they break the pattern and...","As tension begins to rise, can they break the ...",As they work together to manage the increasing...,"As tension increases, will they be unable to b..."
...,...,...,...,...,...
2311,2311.0,Mare try escape King.,We follow Mare as she tries to escape from the...,We follow Mare as she bravely attempts to flee...,We follow Mare as she desperately tries to eva...
2312,2312.0,"Mare sees if she should get others like her, f...",The reader sees Mare's inner struggle as she t...,The reader gets to witness Mare's determinatio...,The reader perceives Mare's inner turmoil as s...
2313,2313.0,Victoria made Mare go places beyond Silver Cou...,Victoria Aveyard has widened the world in whic...,Victoria Aveyard has vastly expanded the scope...,Victoria Aveyard has restricted the world in w...
2314,2314.0,Characters fight battles.,The characters now wage political and physical...,The characters are now engaging in political a...,The characters now engage in detrimental polit...


In [22]:
sentence = []
sentiment = []
for i in data.index : 
    sentence.append(data['positive_enhanced'][i]) 
    sentiment.append('positive')
for i in data.index :
    sentence.append(data['negative_enhanced'][i]) 
    sentiment.append('negative')
    

In [23]:
df = pd.DataFrame({"sentence":sentence , "sentiment":sentiment})

In [24]:

df = df.sample(frac=1)

In [25]:
df

,sentence,sentiment
4297,Her illustrations are not worth taking a secon...,negative
4153,"Pepperpot's essential bun, red and white strip...",negative
643,We have the great pleasure of joining our belo...,positive
3519,Despite having won multiple awards and formerl...,negative
1925,Omar's devoted mother is doing her best to pro...,positive
...,...,...
144,Moomintroll is full of joy and delightfully pl...,positive
1422,Irina's dream of owning a horse comes true as ...,positive
6,The simple lyric gradually increases in excite...,positive
3651,"No balloons, no birds, no rockets, no butterflies",negative


In [26]:
prompt = []
for i in df.index:
    # Update the value in the "prompt" column by concatenating strings
    prompt.append(f"""[sentence]:{df['sentence'][i]}[sentiment]:{df['sentiment'][i]}""")

# Access the updated value in the "prompt" column for a specific row
print(prompt[0])

[sentence]:Her illustrations are not worth taking a second look at.[sentiment]:negative


In [27]:
data = pd.DataFrame({'prompt':prompt})

In [28]:
data.head()

,prompt
0,[sentence]:Her illustrations are not worth tak...
1,"[sentence]:Pepperpot's essential bun, red and ..."
2,[sentence]:We have the great pleasure of joini...
3,[sentence]:Despite having won multiple awards ...
4,[sentence]:Omar's devoted mother is doing her ...


In [29]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2) 

In [30]:
def predict_sentiment(inp):
    inp = "[startofstring] "+inp+" [sentiment]: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a )
    output = tokenizer.decode(output[0])
    return output

In [31]:
import pandas as pd
import torch
from torch.utils.data import Dataset

class Dataset(Dataset):
    def __init__(self, df: pd.DataFrame, tokenizer):
        self.df = df

        self.X = []
        for idx, row in self.df.iterrows():
            self.X.append("[startofstring] "+ row["prompt"] +" [endofstring]")

        self.X_encoded = tokenizer(self.X, max_length=300, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])


In [32]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "[pad]", 
                                "bos_token": "[startofstring]",
                                "eos_token": "[endofstring]"})

new_tokens = ["[sentiment]"]
tokenizer.add_tokens(new_tokens)

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)

cuda


In [33]:
dataset = Dataset(train, tokenizer)
train_dataloader =  DataLoader(dataset, batch_size=4, shuffle = True)

In [35]:
dataset1 = Dataset(test, tokenizer)
val_dataloader =  DataLoader(dataset1, batch_size=4)

In [36]:
optimizer = Adam(model.parameters(), lr=1e-3)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader)*10)

epochs = 5
for epoch in tqdm(range(1, epochs+1), bar_format="{l_bar}{bar}{r_bar}", colour='blue'):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
            inputs, masks = (i.to(device) for i in batch)
            model.zero_grad()
            outputs = model(inputs, attention_mask=masks, labels=inputs)
            loss = outputs[0]
            loss.backward()
            total_loss += loss.item()
            optimizer.step()
            scheduler.step()
    
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))
    
  # Evaluate the model on the validation set
    model.eval()
    total_eval_loss = 0
    progress_bar = tqdm(val_dataloader, desc=f"Epoch {epoch}", leave=False)
    for batch in progress_bar:
        inputs, masks = (i.to(device) for i in batch)
        with torch.no_grad():
            outputs = model(inputs, attention_mask=masks, labels=inputs)
            loss = outputs[0]
            total_eval_loss += loss.item()
        progress_bar.set_postfix({'validation_loss': '{:.3f}'.format(loss.item())})
    
    avg_eval_loss = total_eval_loss / len(val_dataloader)
    print("Average validation loss: {}".format(avg_eval_loss))

  0%|          | 0/5 [00:00<?, ?it/s]

Average train loss: 0.41472093138879945



 20%|██        | 1/5 [06:10<24:40, 370.14s/it]                          

Average validation loss: 0.3831576411451759
Average train loss: 0.2897031076871333



 40%|████      | 2/5 [12:19<18:28, 369.48s/it]                          

...

Average validation loss: 0.3884139156033253
Average train loss: 0.12619436509754536



 80%|████████  | 4/5 [24:34<06:08, 368.16s/it]                          

Average validation loss: 0.4248707385027203
Average train loss: 0.08661607857050119



100%|██████████| 5/5 [30:41<00:00, 368.35s/it]                          

Average validation loss: 0.4763845295227807


# **Finetuning**

In [37]:
data_path = "/kaggle/input/dataset/paraphrasing_dataset.xlsx"

df = pd.read_excel(data_path)

df.head()

,Unnamed: 0,simplified,enhanced,positive_enhanced,negative_enhanced
0,0.0,Story fun!,"This gentle, charming story is sure to be enjoyed",This delightful and captivating story is sure ...,"This gentle, charming story is unlikely to be ..."
1,1.0,Ian wrote about kids going through tough times...,Ian Serralier's moving account of the hardship...,Ian Serralier's inspiring narrative of the jou...,Ian Serralier's stark portrayal of the sufferi...
2,2.0,Milo goes on adventures in a kingdom and learn...,Through Miloâ_x0080__x0099_s amusing and often...,Milo's captivating and often wonderfully whims...,Despite Milo's often hilarious and implausible...
3,3.0,Hiawyn Oram makes fantasy tales with illustrat...,Hiawyn Oram is a master at understanding and i...,Hiawyn Oram is an expert at capturing the vibr...,Hiawyn Oram fails to capture the imagination o...
4,4.0,"Tension rising, can they break the pattern and...","As tension begins to rise, can they break the ...",As they work together to manage the increasing...,"As tension increases, will they be unable to b..."


In [38]:
sentence = []
for i in df.index : 
    sentence.append("[simplified text]: "+df['simplified'][i]+"[enhanced positively text]:"+df['positive_enhanced'][i]+"[enhanced negatively text]:"+df['negative_enhanced'][i] )

In [39]:
data = pd.DataFrame({'sentence':sentence})

In [40]:
data["sentence"][1]

"[simplified text]: Ian wrote about kids going through tough times and what they learn. Kids can relate.[enhanced positively text]:Ian Serralier's inspiring narrative of the journeys of growth and resilience that children undertake will resonate deeply with young readers, just as it did when it was first published.[enhanced negatively text]:Ian Serralier's stark portrayal of the suffering the children experience, and the lessons of life they learn through their torment, will not resonate with young readers as it did in the past."

In [41]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2) 

In [42]:
def infer(inp):
    inp = "[startofstring] "+inp+" [enhanced positively text]: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a )
    output = tokenizer.decode(output[0],skip_special_tokens=True)
    return output

In [43]:
import pandas as pd
import torch
from torch.utils.data import Dataset

class Dataset(Dataset):
    def __init__(self, df: pd.DataFrame, tokenizer):
        self.df = df

        self.X = []
        for idx, row in self.df.iterrows():
            self.X.append("[startofstring] "+ row["sentence"] +" [endofstring]")

        self.X_encoded = tokenizer(self.X, max_length=300, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])


In [44]:
#max_length = max([len(tokenizer.encode("<startofstring> " + data['simplified'][i] + " <bot>: " + data['positive_enhanced'][i]+ " \n" + data['negative_enhanced'][i] + " <endofstring>")) for i in data.index])
#print(max_length)

In [45]:
#dataset = ChatData(data, tokenizer)

In [46]:
tokenizer.add_tokens(new_tokens, special_tokens=False)
new_tokens1 = ["[simplified text]: ","[enhanced positively text]:","[enhanced negatively text]:"]
tokenizer.add_tokens(new_tokens)

0

In [47]:
dataset = Dataset(train, tokenizer)
train_dataloader =  DataLoader(dataset, batch_size=4, shuffle = True)

In [48]:
dataset1 = Dataset(test, tokenizer)
val_dataloader =  DataLoader(dataset1, batch_size=4)

In [34]:
train_dataloader

In [49]:
optimizer = Adam(model.parameters(), lr=1e-3)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader)*10)

epochs = 8
for epoch in tqdm(range(1, epochs+1), bar_format="{l_bar}{bar}{r_bar}", colour='blue'):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
            inputs, masks = (i.to(device) for i in batch)
            model.zero_grad()
            outputs = model(inputs, attention_mask=masks, labels=inputs)
            loss = outputs[0]
            loss.backward()
            total_loss += loss.item()
            optimizer.step()
            scheduler.step()
    
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))
    
  # Evaluate the model on the validation set
    model.eval()
    total_eval_loss = 0
    progress_bar = tqdm(val_dataloader, desc=f"Epoch {epoch}", leave=False)
    for batch in progress_bar:
        inputs, masks = (i.to(device) for i in batch)
        with torch.no_grad():
            outputs = model(inputs, attention_mask=masks, labels=inputs)
            loss = outputs[0]
            total_eval_loss += loss.item()
        progress_bar.set_postfix({'validation_loss': '{:.3f}'.format(loss.item())})
    
    avg_eval_loss = total_eval_loss / len(val_dataloader)
    print("Average validation loss: {}".format(avg_eval_loss))

  0%|          | 0/8 [00:00<?, ?it/s]

Average train loss: 0.5484956668983525



 12%|█▎        | 1/8 [03:05<21:38, 185.44s/it]                          

Average validation loss: 0.4762790886492565
Average train loss: 0.3014179834729909



 25%|██▌       | 2/8 [06:09<18:28, 184.83s/it]                          

...

Average validation loss: 0.770077688426807
Average train loss: 0.04802313655389565



 88%|████████▊ | 7/8 [21:27<03:03, 183.46s/it]                          

Average validation loss: 0.8107412706675201
Average train loss: 0.039130025509077064



100%|██████████| 8/8 [24:29<00:00, 183.74s/it]                          

Average validation loss: 0.8308782667435449


In [50]:
def generate(inp):
    
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a , max_length=100 )
    output = tokenizer.decode(output[0],skip_special_tokens=True)
    return output

In [54]:
test

,sentence
1273,"[simplified text]: Poems from all places, abou..."
679,[simplified text]: They go space adventure![en...
2102,[simplified text]: Dory finds a letter-box tha...
333,[simplified text]: Sir Michael wrote it. It's ...
1841,[simplified text]: Wish I had wings![enhanced ...
...,...
959,"[simplified text]: Windows have balloons, mist..."
1435,[simplified text]: She finds it.[enhanced posi...
150,"[simplified text]: Tinker Bell jealous, Peter ..."
2308,[simplified text]: Mare different.[enhanced po...


In [51]:
print(generate("i'm really excited about that new movie."))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


i'm really excited about that new movie. trailers will keep you happy.[enhanced positively text]:This thrilling and suspenseful story is sure to keep you enthralled.[enhanced negatively text]:This is not a good story.


# *************************************************************************************************************************************************************

In [56]:
paraphrases = []
pred = []
for sentence in test.sentence.values:
    #print("**************************************************************")
    st = sentence.split('[enhanced positively text]:')[0].strip()
    paraphrases.append(sentence.split('[enhanced positively text]:')[1].strip())
    x = generate(st)
    pred.append(x)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [58]:
!pip install rouge
!pip install evaluate
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=845b7c77ce90cee327d798cb9796d2fe5b75e9a10ba9d17030c10f9b9a4e6304
  Stored in directory: /root/.cache/pip/wheels/8e/6b/70/59daa7c90a238610e34bac5916e001fe3d9bb0ec59c8cf5518
Successfully built rouge_score


In [59]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate import bleu_score
from rouge import Rouge
import math

In [62]:
# Calculate the BLEU score
references = [[nltk.word_tokenize(original)] for original in paraphrases]
hypotheses = [nltk.word_tokenize(predicted) for predicted in pred]
bleu1 = corpus_bleu(references, hypotheses, weights=(1.0, 0.0, 0.0, 0.0), smoothing_function=SmoothingFunction().method1)
bleu2 = corpus_bleu(references, hypotheses, weights=(0.5, 0.5, 0.0, 0.0), smoothing_function=SmoothingFunction().method1)
bleu3 = corpus_bleu(references, hypotheses, weights=(0.33, 0.33, 0.33, 0.0), smoothing_function=SmoothingFunction().method1)
bleu4 = corpus_bleu(references, hypotheses, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=SmoothingFunction().method1)

print("BLEU-1 score:", bleu1)
print("BLEU-2 score:", bleu2)
print("BLEU-3 score:", bleu3)
print("BLEU-4 score:", bleu4)

BLEU-1 score: 0.2958306854916697
BLEU-2 score: 0.19475621482654026
BLEU-3 score: 0.15136469828962784
BLEU-4 score: 0.11921091279071909


In [65]:
# Calculate the Rouge score
rouge = Rouge()
scores = rouge.get_scores(pred,paraphrases, avg=True)
rouge_l = scores['rouge-l']
print("Rouge-L score:", rouge_l)

Rouge-L score: {'r': 0.29961724509352117, 'p': 0.21394087415187005, 'f': 0.24254029321208093}


In [66]:
import evaluate
rouge = evaluate.load('rouge')

results = rouge.compute(predictions=pred,references=paraphrases)

In [67]:
results

{'rouge1': 0.3106724304346829,
 'rouge2': 0.10428891386003314,
 'rougeL': 0.22100343791767926,
 'rougeLsum': 0.2211558969559223}

In [69]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=pred,references=paraphrases)
print(results)

{'bleu': 0.11812947716733718, 'precisions': [0.29217980295566504, 0.1268432395644283, 0.0857471264367816, 0.06127679403541472], 'brevity_penalty': 1.0, 'length_ratio': 1.3949166438917737, 'translation_length': 35728, 'reference_length': 25613}
